In [ ]:
from datetime import datetime
import re
import pandas as pd
from config.ai import ModelProvider
from core.analysis import analyze_stock, analyze_stock_no_parse
from core.news import stock_rank, update_ranked_stock_price
from crawlers.general import search_engine
from crawlers.tonghuashun.main import fetch_investment_calendar, realtime_news
from crawlers.tonghuashun.tonghuashun import get_SSE_datas, get_favored_sectors, get_glamour_stocks
from maintenance.init_action import init_stock_market_info
from maintenance.manual import batch_analyze_ranked_stock
from services.ai_service import ai_search, cite_update
from services.news_service import get_stock_news, stock_analyze_prompt
# import services.tushare as ts
from services.tushare import trade_calendar, find_stock_name, get_last_trade_date, get_trade_date_range, realtime_quote, stock_daily_basic, stock_performance, stock_price, news_trade_date
from utils.common import get_date_range, get_today, parse_stock_suggesion, time_now
from utils.db_utils import drop_collection, find_collection_data, store_df_to_mongodb, update_by_id
import logging
from pprint import pprint

log = logging.getLogger(__name__)

In [ ]:

def copy_analyze():
    list_old = find_collection_data('stock_rank_0210_s_01',{},{'_id':0, 'rating': 0, 'stock': 0})
    list_new = find_collection_data('stock_rank_0210_s_02')
    print(list_new[0])
    df_old = pd.DataFrame(list_old)
    df_new = pd.DataFrame(list_new)
    df_new = pd.merge(df_new, df_old,  on="ts_code", how='inner')
    update_by_id(df_new, "stock_rank_0210_s_02")

def transfer():
    drop_collection('stock_rank_0210_s_01')
    data = find_collection_data('stock_rank_0210_s_02')
    store_df_to_mongodb(pd.DataFrame(data), 'stock_rank_0210_s_01')
    drop_collection('stock_rank_0210_s_02')


In [ ]:
# 用交易日前一天的新闻验证当天的股价
# 新闻表格当天应该收录的是前一天15:00到当天15:00的新闻
if 0:
    update_ranked_stock_price('20250211', '20250212')


In [ ]:
# 获取同花顺热门股票数据
if 0:
    data_strings = get_glamour_stocks()
    total_rise = 0

    for i in range (1, 10):
            print("{:d}: {:s}\t {:4.1f}%".format(i, data_strings[i-1][0], data_strings[i-1][1]))
            total_rise = total_rise +data_strings[i-1][1]
    average = total_rise/9
    if(average >= 0):
            print("+{:4.1f} %".format(average))
    else:
            print("{:4.1f}%".format(average))

In [ ]:
if 0:
    data_strings = get_favored_sectors()

    for i in range(1, len(data_strings)):
            print("{:d}: {:s}".format(i, data_strings[i-1]))

In [ ]:
res = trade_calendar(('20250210', '20250220'))
log.info(res)



In [ ]:
from services.tushare import news_collection_name, get_trade_date_range

date = get_trade_date_range(1)[1]
# date = news_collection_name()
print(date)

In [ ]:
from datetime import datetime

from services.tushare import is_date_open, trade_calendar
res = is_date_open('20250214')
df_calendar = trade_calendar(('20250214', '20250214'))

print(res)

In [ ]:
from services.tushare import trade_date_by_given_date, news_collection_name
from datetime import datetime
now = datetime.strptime('20250213 00:00:00', '%Y%m%d %H:%M:%S')
name = news_collection_name(now)
print(name)

In [ ]:
from main import main
main()

In [ ]:
groups = {}
if 'news_0217' not in groups.keys():
    groups['news_0217'] = ['hhaha']
else:
    groups['news_0217'].append(['hhaha'])
print(groups)

In [2]:
index_url = 'https://data.10jqka.com.cn/ifmarket/lhbhistory/orgcode/83ab38bef23d4d98'
orgcode = index_url.split("/")[-1]
print(orgcode)

83ab38bef23d4d98


In [ ]:
from crawlers.tonghuashun.tonghuashun import parse_yyb


